In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
low_memory=False
import pandas as pd

In [ ]:
# Store the path in variables
airlines_path = "airlines.csv"
airport_path = "airports.csv"
flights_path = "flights.csv"

# Load the data
airlines_data = pd.read_csv(airlines_path)
airport_data = pd.read_csv(airport_path)
flights_data = pd.read_csv(flights_path)

In [ ]:
Flight_data_delay =[]
for row in flights_data['ARRIVAL_DELAY']:
    if row > 60:
        Flight_data_delay.append(3)
    elif row > 30:
        Flight_data_delay.append(2)
    elif row > 15:
        Flight_data_delay.append(1)
    else:
        Flight_data_delay.append(0) 

In [ ]:
flights_data['Delay'] = Flight_data_delay


In [ ]:
# flights_data=flights_data.drop(['YEAR'],
#                                              axis=1)

flights_data=flights_data.drop(['YEAR','FLIGHT_NUMBER','AIRLINE','DISTANCE','TAIL_NUMBER','TAXI_OUT','SCHEDULED_TIME','DEPARTURE_TIME','WHEELS_OFF','ELAPSED_TIME','AIR_TIME','WHEELS_ON','DAY_OF_WEEK','TAXI_IN','CANCELLATION_REASON','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', "CANCELLED"],
                                             axis=1)

In [ ]:
flights_data.describe()



In [ ]:
flights_data=flights_data.fillna(flights_data.mean())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
data = flights_data[flights_data['MONTH']==1].values
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
flights_data[flights_data['MONTH']==1]

In [ ]:
data[0]

In [ ]:
encoder = StandardScaler()
encoder.fit(X_train)
scaled_features = encoder.transform(X_train)


In [21]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [22]:
pred_prob = clf.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob, multi_class='ovr')
auc_score

0.9929018081683824

In [23]:
from eurybia import SmartDrift

In [24]:
SD = SmartDrift(df_current=flights_data[flights_data['MONTH']==1],
                df_baseline=flights_data[flights_data['MONTH']==12],
#                 deployed_model=clf, # Optional: put in perspective result with importance on deployed model
#                 encoding=encoder, # Optional: if deployed_model and encoder to use this model
                dataset_names={"df_current": "january", "df_baseline": "october"},
                # Optional: Names for outputs
               )

In [28]:
SD.compile(ignore_cols=['MONTH'], sampling=True)

The following variables are manually set to be ignored in the analysis: 
 ['MONTH']
check and analyze consistency: 0.08892583847045898 sec
df_concat and fill na: 0.18593406677246094 sec
train test split: 0.2041480541229248 sec


/Users/nikp18/opt/anaconda3/envs/housing/lib/python3.8/site-packages/catboost/core.py:1419: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/Users/nikp18/opt/anaconda3/envs/housing/lib/python3.8/site-packages/catboost/core.py:1419: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



train the classifier: 4.907650947570801 sec
compile explainer: 6.779132843017578 sec
explainer feature imp: 6.78646183013916 sec
find df_predict: 6.786787986755371 sec
find roc auc score: 6.805695056915283 sec


/Users/nikp18/opt/anaconda3/envs/housing/lib/python3.8/site-packages/catboost/core.py:1419: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [29]:
import time
st = time.time()
SD.generate_report(    
    output_file='report_flight_delay.html',    
    title_story="Data drift",
    title_description="""flight delay rep""", # Optional: add a subtitle to describe report
    )
print(time.time()-st)

/Users/nikp18/opt/anaconda3/envs/housing/lib/python3.8/site-packages/catboost/core.py:1419: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



Report saved to ./report_flight_delay.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

8.966020107269287
